In [8]:
#!pip install cpi

In [1]:
import numpy as np
import pandas as pd
from siuba import *

from calitp import to_snakecase

import _dla_utils

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enab

In [2]:
import cpi

In [3]:
df= pd.read_parquet("dla_df.parquet")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20970 entries, 0 to 21575
Data columns (total 40 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   location                  20970 non-null  object        
 1   prefix                    20970 non-null  object        
 2   project_no                20970 non-null  object        
 3   agency                    20970 non-null  object        
 4   prepared_date             20956 non-null  datetime64[ns]
 5   submit__to_hq_date        20859 non-null  datetime64[ns]
 6   hq_review_date            20810 non-null  datetime64[ns]
 7   submit_to_fhwa_date       20782 non-null  datetime64[ns]
 8   to_fmis_date              20780 non-null  datetime64[ns]
 9   fed_requested             20969 non-null  float64       
 10  ac_requested              20970 non-null  float64       
 11  total_requested           20969 non-null  float64       
 12  status_comment    

In [5]:
df= df>>select(_.primary_agency_name, _.dist, _.prepared_date, _.prefix, _.prepared_y, _.fed_requested, _.ac_requested, _.total_requested)
df

,primary_agency_name,dist,prepared_date,prefix,prepared_y,fed_requested,ac_requested,total_requested
0,Humboldt County,1,2018-12-18,BPMP,"2,018.00",0.00,0.00,0.00
1,Mendocino County,1,2018-12-17,ER,"2,018.00","11,508.00",0.00,"13,000.00"
2,Humboldt County,1,2018-12-07,ER,"2,018.00","45,499.64",0.00,"51,394.58"
3,Sacramento County,3,2018-12-11,CML,"2,018.00","207,002.00",0.00,"247,002.00"
4,Sacramento County,3,2018-12-05,CML,"2,018.00",0.00,"5,680,921.00","5,702,041.00"
...,...,...,...,...,...,...,...,...
21570,Caltrans,11,NaT,HSIPL,NaN,"351,200.00",0.00,"351,200.00"
21572,Santa Clara Valley Transportation Authority,4,2021-11-22,FTASTPL,"2,021.00","1,987,000.00",0.00,"1,987,000.00"
21573,Alameda - Contra Costa Transit District,4,2021-11-22,FTASTPL,"2,021.00","1,254,000.00",0.00,"1,254,000.00"
21574,Mountain Area Regional Transit Authority,8,2022-02-18,FTACML,"2,022.00","438,168.00",0.00,"438,168.00"


In [6]:
sorted(df.prepared_y.unique())

[2010.0,
 2011.0,
 2012.0,
 2013.0,
 2014.0,
 2015.0,
 2016.0,
 2017.0,
 2018.0,
 nan,
 2019.0,
 2020.0,
 2021.0,
 2022.0]

In [7]:
df.tail(1)

,primary_agency_name,dist,prepared_date,prefix,prepared_y,fed_requested,ac_requested,total_requested
21575,San Diego Metropolitan Transit System,11,2022-01-25,FTASTPL,"2,022.00","10,000,000.00",0.00,"10,000,000.00"


In [8]:
cpi.inflate(351200, 2018)

378980.5302122203

In [9]:
# Inflation table
def inflation_table(base_year):
    cpi.update()
    series_df = cpi.series.get(area="U.S. city average").to_dataframe()
    inflation_df = (series_df[series_df.year >= 2008]
           .pivot_table(index='year', values='value', aggfunc='mean')
           .reset_index()
          )
    denominator = inflation_df.value.loc[inflation_df.year==base_year].iloc[0]

    inflation_df = inflation_df.assign(
        inflation = inflation_df.value.divide(denominator)
    )
    
    return inflation_df

In [10]:
inflation_table(2021)

,year,value,inflation
0,2008,215.30,0.79
1,2009,214.54,0.79
2,2010,218.06,0.80
3,2011,224.94,0.83
4,2012,229.59,0.85
5,2013,232.96,0.86
6,2014,236.74,0.87
7,2015,237.02,0.87
8,2016,240.01,0.89
9,2017,245.12,0.90


In [11]:
def adjust_prices(df):
    
    cols =  ["total_requested",
           "fed_requested",
           "ac_requested"]
    
    ##get cpi table 
    cpi = inflation_table(2021)
    cpi.update
    cpi = (cpi>>select(_.year, _.value))
    cpi_dict = dict(zip(cpi['year'], cpi['value']))
    
    
    for col in cols:
        multiplier = df["prepared_y"].map(cpi_dict)  
    
        ##using 270.97 for 2021 dollars
        df[f"adjusted_{col}"] = ((df[col] * 270.97) / multiplier)
    return df


In [12]:
adjust_prices(df)

,primary_agency_name,dist,prepared_date,prefix,prepared_y,fed_requested,ac_requested,total_requested,adjusted_total_requested,adjusted_fed_requested,adjusted_ac_requested
0,Humboldt County,1,2018-12-18,BPMP,"2,018.00",0.00,0.00,0.00,0.00,0.00,0.00
1,Mendocino County,1,2018-12-17,ER,"2,018.00","11,508.00",0.00,"13,000.00","14,028.33","12,418.31",0.00
2,Humboldt County,1,2018-12-07,ER,"2,018.00","45,499.64",0.00,"51,394.58","55,460.01","49,098.77",0.00
3,Sacramento County,3,2018-12-11,CML,"2,018.00","207,002.00",0.00,"247,002.00","266,540.45","223,376.35",0.00
4,Sacramento County,3,2018-12-05,CML,"2,018.00",0.00,"5,680,921.00","5,702,041.00","6,153,086.12",0.00,"6,130,295.48"
...,...,...,...,...,...,...,...,...,...,...,...
21570,Caltrans,11,NaT,HSIPL,NaN,"351,200.00",0.00,"351,200.00",NaN,NaN,NaN
21572,Santa Clara Valley Transportation Authority,4,2021-11-22,FTASTPL,"2,021.00","1,987,000.00",0.00,"1,987,000.00","1,987,001.69","1,987,001.69",0.00
21573,Alameda - Contra Costa Transit District,4,2021-11-22,FTASTPL,"2,021.00","1,254,000.00",0.00,"1,254,000.00","1,254,001.07","1,254,001.07",0.00
21574,Mountain Area Regional Transit Authority,8,2022-02-18,FTACML,"2,022.00","438,168.00",0.00,"438,168.00","422,305.63","422,305.63",0.00


In [13]:
df>>filter(_.prepared_y.isnull())

,primary_agency_name,dist,prepared_date,prefix,prepared_y,fed_requested,ac_requested,total_requested,adjusted_total_requested,adjusted_fed_requested,adjusted_ac_requested
4330,Sunline Transit Agency,7,NaT,HSIMG,NaN,"829,741.26",0.00,"829,741.26",NaN,NaN,NaN
5234,Sunline Transit Agency,7,NaT,BRIM,NaN,0.00,0.00,0.00,NaN,NaN,NaN
5257,Sunline Transit Agency,7,NaT,BRIM,NaN,0.00,0.00,-0.05,NaN,NaN,NaN
5712,Sunline Transit Agency,7,NaT,BRIM,NaN,-0.06,0.00,-0.06,NaN,NaN,NaN
5849,Sunline Transit Agency,7,NaT,BRIM,NaN,"192,176.47",0.00,"174,090.31",NaN,NaN,NaN
18071,"U.S. Forest Service, Pacific Southwest Region",3,NaT,USFSSTPL,NaN,"308,000.00",0.00,"308,000.00",NaN,NaN,NaN
18072,"U.S. Forest Service, Pacific Southwest Region",3,NaT,USFSSTPL,NaN,"220,889.00",0.00,"220,889.00",NaN,NaN,NaN
18073,"U.S. Forest Service, Pacific Southwest Region",3,NaT,FTA,NaN,"220,889.00",0.00,"220,889.00",NaN,NaN,NaN
18074,San Francisco Bay Area Water Transit Authority,4,NaT,FTAFBP,NaN,"1,356,699.00",0.00,"1,356,699.00",NaN,NaN,NaN
18075,San Francisco Bay Area Water Transit Authority,4,NaT,FTAFBP,NaN,"1,664,572.00",0.00,"1,664,572.00",NaN,NaN,NaN


In [14]:
df1= df>>filter(_.prepared_y<=2021)

In [15]:
df1["real_total_dollars"] = df1.apply(lambda x: cpi.inflate(x["total_requested"], x["prepared_date"]) if pd.notnull(x['prepared_date']) else None, axis=1)

/tmp/ipykernel_904/1104328080.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [16]:
df1

,primary_agency_name,dist,prepared_date,prefix,prepared_y,fed_requested,ac_requested,total_requested,adjusted_total_requested,adjusted_fed_requested,adjusted_ac_requested,real_total_dollars
0,Humboldt County,1,2018-12-18,BPMP,"2,018.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,Mendocino County,1,2018-12-17,ER,"2,018.00","11,508.00",0.00,"13,000.00","14,028.33","12,418.31",0.00,"14,547.95"
2,Humboldt County,1,2018-12-07,ER,"2,018.00","45,499.64",0.00,"51,394.58","55,460.01","49,098.77",0.00,"57,514.27"
3,Sacramento County,3,2018-12-11,CML,"2,018.00","207,002.00",0.00,"247,002.00","266,540.45","223,376.35",0.00,"276,413.20"
4,Sacramento County,3,2018-12-05,CML,"2,018.00",0.00,"5,680,921.00","5,702,041.00","6,153,086.12",0.00,"6,130,295.48","6,380,998.61"
...,...,...,...,...,...,...,...,...,...,...,...,...
21490,San Bernardino,8,2021-08-19,HSIPL,"2,021.00","4,001,400.00",0.00,"4,853,917.00","4,853,921.13","4,001,403.41",0.00,"4,988,427.17"
21506,San Bernardino County,8,2021-06-28,STPLER,"2,021.00","1,105,000.00",0.00,"4,203,285.00","4,203,288.58","1,105,000.94",0.00,"4,349,512.58"
21537,Amador County,10,2021-10-05,ER,"2,021.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00
21572,Santa Clara Valley Transportation Authority,4,2021-11-22,FTASTPL,"2,021.00","1,987,000.00",0.00,"1,987,000.00","1,987,001.69","1,987,001.69",0.00,"2,009,876.22"


In [17]:
from datetime import datetime

In [18]:
cpi.inflate(284963, datetime(2022, 1, 1))

284963.0

In [19]:
df>>filter(_.prepared_y==2015)

,primary_agency_name,dist,prepared_date,prefix,prepared_y,fed_requested,ac_requested,total_requested,adjusted_total_requested,adjusted_fed_requested,adjusted_ac_requested
1010,Lake Elsinore,8,2015-12-02,BRLS,"2,015.00","1,639,576.00",0.00,"1,852,000.00","2,117,301.46","1,874,447.44",0.00
2508,Humboldt County,1,2015-10-12,ER,"2,015.00","1,329,549.14",0.00,"1,501,805.00","1,716,940.56","1,520,008.82",0.00
3219,Manhattan Beach,7,2015-10-15,HPLUL,"2,015.00","481,250.00",0.00,"481,250.00","550,189.70","550,189.70",0.00
3658,Palm Springs,8,2015-12-07,BRLS,"2,015.00","330,217.00",0.00,"373,000.00","426,432.75","377,521.02",0.00
4858,Alameda County,4,2015-10-19,HPLUL,"2,015.00",-0.04,0.00,"22,718.91","25,973.42",-0.05,0.00
...,...,...,...,...,...,...,...,...,...,...,...
10256,Davis,3,2015-01-02,STPL,"2,015.00",0.00,0.00,"16,912.51","19,335.25",0.00,0.00
10257,San Carlos,4,2015-01-02,HSIP,"2,015.00",0.00,0.00,"12,075.00","13,804.76",0.00,0.00
10258,Campbell,4,2015-01-02,CML,"2,015.00",0.00,0.00,"182,395.73","208,524.16",0.00,0.00
10259,Pleasant Hill,4,2015-01-02,STPLX,"2,015.00","-11,726.26",0.00,"-13,245.89","-15,143.38","-13,406.06",0.00


In [20]:
#checking to see if it works. using first row
cpi.inflate(1852000.00, 2015)

2117301.4593889895

In [21]:
df>>filter(_.prepared_y== 2014)

,primary_agency_name,dist,prepared_date,prefix,prepared_y,fed_requested,ac_requested,total_requested,adjusted_total_requested,adjusted_fed_requested,adjusted_ac_requested
8246,Coachella,8,2014-09-25,HPLULN,"2,014.00","719,920.75",0.00,"1,968,000.00","2,252,587.75","824,026.76",0.00
8257,Palm Springs,8,2014-07-03,HPLUL,"2,014.00","399,902.72",0.00,"1,130,683.00","1,294,188.35","457,731.69",0.00
8366,Richmond,4,2014-12-19,CML,"2,014.00",0.00,0.00,"1,130,041.00","1,293,453.51",0.00,0.00
8418,Monterey Park,7,2014-08-14,STPL,"2,014.00","1,036,867.00",0.00,"1,057,741.00","1,210,698.38","1,186,805.84",0.00
8488,Moreno Valley,8,2014-12-10,STPL,"2,014.00","1,084,000.00",0.00,"2,008,167.00","2,298,563.20","1,240,754.63",0.00
...,...,...,...,...,...,...,...,...,...,...,...
12988,Moreno Valley,8,2014-01-06,BRLS,"2,014.00","-131,768.16",0.00,"28,508.61","32,631.17","-150,822.84",0.00
12989,Santa Clarita,7,2014-01-06,BHLS,"2,014.00",-289.94,0.00,-327.50,-374.86,-331.87,0.00
12998,Ridgecrest,9,2014-01-03,STPL,"2,014.00",-198.00,0.00,"20,987.53","24,022.49",-226.63,0.00
13001,Solano County,4,2014-01-02,CML,"2,014.00","60,000.00",0.00,"67,774.00","77,574.64","68,676.46",0.00


In [22]:
#checking to see if it works. using first row
cpi.inflate(1968000, 2014)

2252589.2133008926